In [34]:
function my_function()
    # 在函数中执行一些操作
    x = 0
    for i in 1:100000000
        x = i^2
    end
    return x
end

# 设置最大时间为10秒
max_time = 1e-9

function verify_an_instance()
    # 使用一个循环来检查时间是否超过最大时间
    while true
        # 执行代码
        res = @timed my_function()
        
        # 获取实际执行时间
        actual_time = res.time

        if(actual_time >= max_time)
            print(actual_time)
            print("\n")
            print(11111111)
            print("\n")
            return 0
        end 
        # 如果实际执行时间超过最大时间，就强制停止代码的执行
        #= if actual_time < max_time
            print(actual_time)
            print("\n") 
            return res.value
        end =#
    end
end

result = @timed verify_an_instance()
print(result.value)
#= print("\n")
print(elapsed_time) =#

8.3e-8
11111111
0